In [108]:
import pandas as pd
import numpy as np
from scipy.integrate import simps, trapz, quad

In [99]:
meta_df = pd.read_csv('metadata.csv')

In [100]:
battery_id_df = meta_df['battery_id'].unique()
battery_id_df

array(['B0047', 'B0045', 'B0048', 'B0046', 'B0043', 'B0032', 'B0039',
       'B0040', 'B0029', 'B0028', 'B0042', 'B0034', 'B0038', 'B0033',
       'B0030', 'B0041', 'B0027', 'B0044', 'B0036', 'B0025', 'B0026',
       'B0031', 'B0049', 'B0050', 'B0052', 'B0051', 'B0006', 'B0005',
       'B0007', 'B0018', 'B0053', 'B0054', 'B0056', 'B0055'], dtype=object)

In [101]:
battery_df = meta_df[(meta_df.battery_id == battery_id_df[0]) & (meta_df.type != 'impedance')]
battery_df

,type,start_time,ambient_temperature,battery_id,test_id,uid,filename,Capacity,Re,Rct
0,discharge,[2010. 7. 21. 15. 0. ...,4,B0047,0,1,00001.csv,1.6743047446975208,NaN,NaN
2,charge,[2010. 7. 21. 17. 25. ...,4,B0047,2,3,00003.csv,NaN,NaN,NaN
4,discharge,[2.0100e+03 7.0000e+00 2.1000e+01 2.1000e+01 2...,4,B0047,4,5,00005.csv,1.5243662105099023,NaN,NaN
5,charge,[2010. 7. 21. 22. 38. ...,4,B0047,5,6,00006.csv,NaN,NaN,NaN
6,discharge,[2.010e+03 7.000e+00 2.200e+01 1.000e+00 4.000...,4,B0047,6,7,00007.csv,1.5080762969973425,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
177,charge,[2010 8 17 2 32 20],4,B0047,177,178,00178.csv,NaN,NaN,NaN
178,discharge,[2010. 8. 17. 5. 33. ...,4,B0047,178,179,00179.csv,1.158443027293818,NaN,NaN
179,charge,[2010. 8. 17. 6. 50. ...,4,B0047,179,180,00180.csv,NaN,NaN,NaN
180,discharge,[2010. 8. 17. 9. 52. ...,4,B0047,180,181,00181.csv,1.1567087516841796,NaN,NaN


In [102]:
discharge_df = pd.read_csv('./data/' +  battery_df.iloc[0]['filename'])
discharge_df = discharge_df[discharge_df['Current_load'] == 1]
discharge_df['Time'] /= 3600
discharge_df

,Voltage_measured,Current_measured,Temperature_measured,Current_load,Voltage_load,Time
2,4.039277,-0.995093,6.250255,1.0,3.465,0.006467
3,4.019506,-0.996731,6.302176,1.0,3.451,0.010113
4,4.004763,-0.992845,6.361645,1.0,3.438,0.013785
5,3.992863,-0.995957,6.399692,1.0,3.427,0.017448
6,3.982347,-0.994665,6.233613,1.0,3.419,0.021081
...,...,...,...,...,...,...
466,2.593520,-0.993901,11.977146,1.0,2.052,1.701511
467,2.565613,-0.995725,12.038215,1.0,2.025,1.705243
468,2.536253,-0.993844,12.128629,1.0,1.996,1.708967
469,2.504669,-0.994458,12.234626,1.0,1.965,1.712687


In [103]:
time = discharge_df['Time'].values # 초
current = discharge_df['Current_measured'].values # 전류

In [107]:
# 쿨롱 계수 계산
def coulomb_integral(time, current, method='trapz'):
    if method == 'trapz':
        capacity = np.abs(trapz(current, time))  # 사다리꼴 방법
    elif method == 'simps':
        capacity = np.abs(simps(current, time))  # 심프슨 방법
    else:
        raise ValueError('Invalid method')
    return capacity

capacity_trapz = coulomb_integral(time, current, method='trapz')
capacity_simps = coulomb_integral(time, current, method='simps') 


print('Trapz method: Capacity = {:.6f} Ahr'.format(capacity_trapz))
print('Simps method: Capacity = {:.6f} Ahr'.format(capacity_simps)) 


Trapz method: Capacity = 1.702005 Ahr
Simps method: Capacity = 1.702006 Ahr


In [105]:
# 구간별 쿨롱 계수 계산
discharge_df['Capacity_trapz'] = 0.0  # 사다리꼴 방법 쿨롱 계수
discharge_df['Capacity_simps'] = 0.0  # 심프슨 방법 쿨롱 계수

for i in discharge_df.index:
    time = discharge_df['Time'][:i+1].to_numpy()
    current = discharge_df['Current_measured'][:i+1].to_numpy()
    
    capacity_trapz = coulomb_integral(time, current, method='trapz')
    capacity_simps = coulomb_integral(time, current, method='simps')
    
    discharge_df.loc[i, 'Capacity_trapz'] = capacity_trapz
    discharge_df.loc[i, 'Capacity_simps'] = capacity_simps
discharge_df

,Voltage_measured,Current_measured,Temperature_measured,Current_load,Voltage_load,Time,Capacity_trapz,Capacity_simps
2,4.039277,-0.995093,6.250255,1.0,3.465,0.006467,0.007284,0.007287
3,4.019506,-0.996731,6.302176,1.0,3.451,0.010113,0.010927,0.010928
4,4.004763,-0.992845,6.361645,1.0,3.438,0.013785,0.014542,0.014548
5,3.992863,-0.995957,6.399692,1.0,3.427,0.017448,0.018180,0.018185
6,3.982347,-0.994665,6.233613,1.0,3.419,0.021081,0.021809,0.021820
...,...,...,...,...,...,...,...,...
466,2.593520,-0.993901,11.977146,1.0,2.052,1.701511,1.698292,1.698294
467,2.565613,-0.995725,12.038215,1.0,2.025,1.705243,1.702005,1.702006
468,2.536253,-0.993844,12.128629,1.0,1.996,1.708967,1.702005,1.702006
469,2.504669,-0.994458,12.234626,1.0,1.965,1.712687,1.702005,1.702006
